<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# %load "load_libraries_and_datasets.py"
## 1. Load libraries and set env 
import warnings
warnings.filterwarnings('ignore')

## data structures and manipulation 
import pandas as pd
import numpy as np 

## visualization 
import seaborn as sns
import matplotlib.pyplot as plt 

## particular datatypes manipulation 
from dateutil.parser import parse

## statistical analysis - Ho-testing and glm Regression 
import statsmodels.api as sm
import statsmodels.formula.api as smf
## patsy is a Python library for describing statistical models and building Design Matrices using R-like formulas.
import patsy 

import scipy.stats as sci 
import math 


## Shared Env
%matplotlib inline 
from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('pdf', 'png')


## plot styles 
plt.style.use('seaborn-white')
plt.style.use('ggplot') 
sns.set(color_codes=True)


## allow multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Getting R in the mix =D  << TODO: put R libs in a seperate Rscript file and load it from here<<< Excited!!!!!
# %load_ext rpy2.ipython 
# %R require(ggplot2); require(tidyr); require(dplyr) 

In [2]:
### Formatting preambles
#plt.rcParams['savefig.dpi'] = 75

#plt.rcParams['figure.autolayout'] = False
#plt.rcParams['figure.figsize'] = 10, 6
#plt.rcParams['axes.labelsize'] = 18
#plt.rcParams['axes.titlesize'] = 20
#plt.rcParams['font.size'] = 16
#plt.rcParams['lines.linewidth'] = 2.0
#plt.rcParams['lines.markersize'] = 8
#plt.rcParams['legend.fontsize'] = 14

#plt.rcParams['text.usetex'] = True
#plt.rcParams['font.family'] = "serif"
#plt.rcParams['font.serif'] = "cm"
#plt.rcParams['text.latex.preamble'] = "\usepackage{subdepth}, \usepackage{type1cm}"

In [9]:
# help(patsy)
## Functions 
def sampleStructure(dtset, grp):
    argz = { 
        "CHP ID": {"# CHPs": "count"},
        "Households Registered": { "# HHs": "sum", "Avg.CHP.Size": "mean"},
        "Sales" : { "KES Sales": "sum", "Avg.CHP Sales": "mean"},
        "Pregnancy" : { "# Pregoz Cases": "sum", "Avg.CHP Pregoz": "mean"},
        "Community Events" : { "# Events": "sum", "Avg.CHP Events": "mean"}, 
        "All Malaria Assessments" : { "# Malaria Cases": "sum", "Avg.CHP Malaria": "mean"},
        "Total U5 Assessments" : { "# Child Cases": "sum", "Avg.CHP U5": "mean"},
        "Total U1 Assessments" : { "# U1 Cases": "sum", "Avg.CHP U1": "mean"},        
    }
    return dtset.groupby(grp).agg( argz).round(0)


## TODO: with margins + fq% 
def summarizer(dtset): 
    return dtset.describe().round(2)

## TODO: update to sns and with group by
def distributionz(dtset, colz, grpz=""):
    fig, ax = plt.subplots( figsize=(10,10))
    # return dtset.plot( colz, kind="hist", legend=True, alpha=0.4, ax=ax ) 
    return dtset.hist(colz,stacked=True,ax=ax)

def boxplotz(dtset, colz, grp):
    fig, ax = plt.subplots( figsize=(10,10))
    return dtset.boxplot( colz, grp, ax)

def corrz(dtset, colz, mthd="pearson"):
    return dtset[colz].corr( method=mthd).round(3)
    
def plotCorrz( cor_outz ):
    # Generate a mask for the upper triangle
    mask = np.zeros_like(cor_outz, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(7, 7))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    #TODO: pretty print tables : cor_outz, 
    return sns.heatmap(cor_outz, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

## TODO: generalize to smalls 
def quintilize(dtset, col, nbGrpz=5, labelzPrefix="Quint"):
    labelz = []
    for i in range(nbGrpz):
        labelz.append( labelzPrefix+"_"+str(i+1))
    dtset["Quints_"+col] = pd.qcut( dtset[col], nbGrpz, labels=labelz, duplicates='drop') 
    

def checkMissing(dtset):
    #dataset.fillna( np.nan )
    return "\nShape # records, # variables", dtset.shape, "\n\n\n Counts of Missing Values \n", dtset.apply( lambda x: sum(x.isnull() ), axis=0)




